In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

tf.enable_eager_execution()
tf.__version__

'1.14.0'

# Softmax classification

* 임의의 Dataset 준비


In [0]:
x_data = [[1., 2., 1., 1.],
          [2., 1., 3., 2.],
          [3., 1., 3., 4.],
          [4., 1., 5., 5.],
          [1., 7., 5., 5.],
          [1., 2., 5., 6.],
          [1., 6., 6., 6.],
          [1., 7., 7., 7.]] # 8x4
y_data = [[0., 0., 1.],
          [0., 0., 1.],
          [0., 0., 1.],
          [0., 1., 0.],
          [0., 1., 0.],
          [0., 1., 0.],
          [1., 0., 0.],
          [1., 0., 0.]] # 8x3

x_test = [[1., 1., 1., 1.]]
y_test = [[0., 0., 1.]]

#convert into numpy and float format
x_data = np.asarray(x_data, dtype=np.float32)
y_data = np.asarray(y_data, dtype=np.float32)


## 임의의 Data를 이용해서 3개의 클래스를 가지는 데이터셋 생성

In [0]:
#dataset을 선언합니다.
dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data))
dataset = dataset.repeat(1).batch(8)

nb_classes = 3 # class의 개수입니다.

print(x_data.shape)
print(y_data.shape)

(8, 4)
(8, 3)


In [0]:
#Weight and bias setting
W = tf.Variable(tf.random_normal([4, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')
variables = [W, b]

print(W,b)

<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[ 1.0411005 , -1.2035726 , -0.9278547 ],
       [ 0.08599742,  0.97621495,  0.19296075],
       [ 0.09499903, -0.5954188 , -1.0320287 ],
       [ 0.9404571 ,  0.06505298,  0.30034235]], dtype=float32)> <tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([-0.08296263, -0.32026964,  0.14704172], dtype=float32)>


In [0]:
# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
def hypothesis_softmax(X):
    return tf.nn.softmax(tf.matmul(X, W) + b)

print(hypothesis_softmax(x_data))

tf.Tensor(
[[8.7661028e-01 9.0803273e-02 3.2586426e-02]
 [9.9927038e-01 4.7167155e-04 2.5788791e-04]
 [9.9998128e-01 8.6842711e-06 1.0014675e-05]
 [9.9999976e-01 9.6387225e-08 7.7377841e-08]
 [9.8331439e-01 1.6632548e-02 5.3131189e-05]
 [9.9990106e-01 8.2216204e-05 1.6685848e-05]
 [9.9854290e-01 1.4487343e-03 8.2815232e-06]
 [9.9926072e-01 7.3771703e-04 1.5754899e-06]], shape=(8, 3), dtype=float32)


## 가설을 검증할 Cost 함수를 정의합니다
$$
\begin{align}
cost(h(x),y) & = −log(h(x))  &  if  &  y=1 \\\\\
cost(h(x),y) & = -log(1−h(x))  &  if  &  y=0
\end{align}
$$

### 두 식을 한번에 쓰게되면,

$$
\begin{align}
cost(h(x),y) & = −y log(h(x))−(1−y)log(1−h(x))
\end{align}
$$

In [0]:
def loss_fn(hypothesis, labels):
    cost = -tf.reduce_mean(labels * tf.log(hypothesis) + (1 - labels) * tf.log(1 - hypothesis))
    return cost

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)

In [0]:
epochs = 5000

for step in range(epochs):
  for features, labels in dataset:
    with tf.GradientTape() as tape:
      loss_value = loss_fn(hypothesis_softmax(features),labels)
      grads = tape.gradient(loss_value, [W,b])
      optimizer.apply_gradients(grads_and_vars=zip(grads,[W,b]))
      if step % 100 == 0:
            print("Iter: {}, Loss: {:.4f}".format(step, loss_fn(hypothesis_softmax(features),labels)))

Iter: 0, Loss: 0.4946
Iter: 100, Loss: 0.2212
Iter: 200, Loss: 0.2191
Iter: 300, Loss: 0.2182
Iter: 400, Loss: 0.2173
Iter: 500, Loss: 0.2166
Iter: 600, Loss: 0.2159
Iter: 700, Loss: 0.2152
Iter: 800, Loss: 0.2146
Iter: 900, Loss: 0.2139
Iter: 1000, Loss: 0.2133
Iter: 1100, Loss: 0.2126
Iter: 1200, Loss: 0.2120
Iter: 1300, Loss: 0.2114
Iter: 1400, Loss: 0.2108
Iter: 1500, Loss: 0.2102
Iter: 1600, Loss: 0.2096
Iter: 1700, Loss: 0.2090
Iter: 1800, Loss: 0.2084
Iter: 1900, Loss: 0.2078
Iter: 2000, Loss: 0.2073
Iter: 2100, Loss: 0.2067
Iter: 2200, Loss: 0.2061
Iter: 2300, Loss: 0.2056
Iter: 2400, Loss: 0.2050
Iter: 2500, Loss: 0.2044
Iter: 2600, Loss: 0.2039
Iter: 2700, Loss: 0.2033
Iter: 2800, Loss: 0.2028
Iter: 2900, Loss: 0.2023
Iter: 3000, Loss: 0.2017
Iter: 3100, Loss: 0.2012
Iter: 3200, Loss: 0.2007
Iter: 3300, Loss: 0.2002
Iter: 3400, Loss: 0.1996
Iter: 3500, Loss: 0.1991
Iter: 3600, Loss: 0.1986
Iter: 3700, Loss: 0.1981
Iter: 3800, Loss: 0.1976
Iter: 3900, Loss: 0.1971
Iter: 4000, 

In [0]:
sample_data = [[2,1,3,2]] # answer_label [[0,0,1]]
sample_data = np.asarray(sample_data, dtype=np.float32)

a = hypothesis_softmax(sample_data)

print(a)
print(tf.argmax(a, 1)) #index: 2

tf.Tensor(
[[0.02506327 0.17484154 0.8000952 ]
 [0.02767292 0.21975704 0.7525701 ]
 [0.02901728 0.26688847 0.70409423]
 [0.03237214 0.46170443 0.50592345]
 [0.06300007 0.44261736 0.49438262]
 [0.04512385 0.5282038  0.42667234]
 [0.0807696  0.4135782  0.5056521 ]
 [0.09640567 0.37714964 0.52644473]], shape=(8, 3), dtype=float32)
tf.Tensor([2 2 2 2 2 1 2 2], shape=(8,), dtype=int64)


In [0]:
b = hypothesis_softmax(x_data)
print(b)
print(tf.argmax(b, 1))
print(tf.argmax(y_data, 1)) # matches with y_data


tf.Tensor(
[[2.8544392e-03 3.4178246e-02 9.6296734e-01]
 [2.7672928e-02 2.1975701e-01 7.5257009e-01]
 [7.5363736e-05 3.0042809e-01 6.9949651e-01]
 [1.5564926e-04 6.8860263e-01 3.1124178e-01]
 [3.7244135e-01 5.7660097e-01 5.0957628e-02]
 [1.7641214e-01 8.2352173e-01 6.6137502e-05]
 [7.0105165e-01 2.9804820e-01 9.0011442e-04]
 [8.4398997e-01 1.5587109e-01 1.3893055e-04]], shape=(8, 3), dtype=float32)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)


In [0]:
def accuracy_fn(hypothesis, labels):
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.float32))
    return accuracy
  
test_acc = accuracy_fn(hypothesis_softmax(x_data),y_data)
print("Testset Accuracy: {:.4f}".format(test_acc))

Testset Accuracy: 1.0000
